In [5]:
import torch
import tensorflow as tf
from langchain_community.document_loaders import PyPDFLoader, TextLoader

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("using GPU:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("using CPU")

using GPU: NVIDIA GeForce RTX 4060 Laptop GPU


In [3]:
from ragatouille import RAGPretrainedModel
RAG = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")

In [6]:
loader = PyPDFLoader("hallucination_paper.pdf")
pages = loader.load_and_split()

In [8]:
len(data_pages)

4602

In [12]:
len(pages)

89

In [9]:
# full_document = ""

# for page in pages:
#   full_document += page.page_content

In [14]:
print(full_document)

A Survey on Hallucination in Large Language Models:
Principles, Taxonomy, Challenges, and Open Questions
Lei Huang1∗, Weijiang Yu2∗, Weitao Ma1, Weihong Zhong1
Zhangyin Feng1, Haotian Wang1, Qianglong Chen2, Weihua Peng2
Xiaocheng Feng1†, Bing Qin1, Ting Liu1
1Harbin Institute of Technology, Harbin, China
2Huawei Inc., Shenzhen, China
{lhuang,wtma,whzhong,zyfeng,xcfeng†,qinb,tliu}@ir.hit.edu.cn
{weijiangyu8,wanght1998,chenqianglong.ai,pengwh.hit}@gmail.com
Abstract
The emergence of large language models
(LLMs) has marked a significant breakthrough
in natural language processing (NLP), leading
to remarkable advancements in text understand-
ing and generation. Nevertheless, alongside
these strides, LLMs exhibit a critical tendency
to produce hallucinations, resulting in content
that is inconsistent with real-world facts or user
inputs. This phenomenon poses substantial
challenges to their practical deployment and
raises concerns over the reliability of LLMs in
real-world scenarios, which

In [19]:
type(full_document)

str

In [20]:
RAG.index(collection=[full_document], 
          index_name="hallucination_paper", 
          max_document_length=512, 
          split_documents=True)

---- WARNING! You are using PLAID with an experimental replacement for FAISS for greater compatibility ----
This is a behaviour change from RAGatouille 0.8.0 onwards.
This works fine for most users and smallish datasets, but can be considerably slower than FAISS and could cause worse results in some situations.
If you're confident with FAISS working on your machine, pass use_faiss=True to revert to the FAISS-using behaviour.
--------------------


[Apr 06, 21:52:27] #> Creating directory .ragatouille/colbert/indexes/hallucination_paper 


[Apr 06, 21:52:30] [0] 		 #> Encoding 139 passages..
[Apr 06, 21:52:33] [0] 		 avg_doclen_est = 366.3165588378906 	 len(local_sample) = 139
[Apr 06, 21:52:33] [0] 		 Creating 2,048 partitions.
[Apr 06, 21:52:33] [0] 		 *Estimated* 50,918 embeddings.
[Apr 06, 21:52:33] [0] 		 #> Saving the indexing plan to .ragatouille/colbert/indexes/hallucination_paper/plan.json ..
used 20 iterations (0.7326s) to cluster 48373 items into 2048 clusters
[Apr 06, 21:52:

0it [00:00, ?it/s]

[Apr 06, 21:53:58] [0] 		 #> Encoding 139 passages..


1it [00:02,  2.06s/it]
100%|███████████████████████| 1/1 [00:00<00:00, 85.84it/s]

[Apr 06, 21:54:00] #> Optimizing IVF to store map from centroids to list of pids..
[Apr 06, 21:54:00] #> Building the emb2pid mapping..
[Apr 06, 21:54:00] len(emb2pid) = 50918



100%|██████████████| 2048/2048 [00:00<00:00, 89676.52it/s]

[Apr 06, 21:54:00] #> Saved optimized IVF to .ragatouille/colbert/indexes/hallucination_paper/ivf.pid.pt
Done indexing!


'.ragatouille/colbert/indexes/hallucination_paper'

In [25]:
with torch.device(device):
    search = RAG.search(query="What is Hallucination?", k=3)

Loading searcher for index hallucination_paper for the first time... This may take a few seconds
[Apr 06, 22:01:57] #> Loading codec...
[Apr 06, 22:01:57] #> Loading IVF...
[Apr 06, 22:01:57] #> Loading doclens...


100%|██████████████████████| 1/1 [00:00<00:00, 285.31it/s]

[Apr 06, 22:01:57] #> Loading codes and residuals...



100%|███████████████████████| 1/1 [00:00<00:00, 15.02it/s]

Searcher loaded!

#> QueryTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
#> Input: . What is Hallucination?, 		 True, 		 None
#> Output IDs: torch.Size([32]), tensor([  101,     1,  2054,  2003,  2534, 14194, 12758,  1029,   102,   103,
          103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
          103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
          103,   103], device='cuda:0')
#> Output Mask: torch.Size([32]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')



In [1]:
# search

## Using Langchain Retrieval

In [27]:
retriever = RAG.as_langchain_retriever(k=3)

In [28]:
retriever.invoke("What is hallucination?")

[Document(page_content='Such\nintegration of human feedback into the training\nloop has proven effective in enhancing the align-\nment of LLMs, guiding them toward producing\nhigh-quality and harmless responses.\n2.3 Hallucinations in Large Language Models\nThe concept of hallucination traces its roots to the\nfields of pathology and psychology and is defined\nasthe perception of an entity or event that is ab-\nsent in reality (Macpherson and Platchias, 2013).\nWithin the realm of NLP, hallucination is typically\nreferred to as a phenomenon in which the gen-\nerated content appears nonsensical or unfaithful\nto the provided source content (Filippova, 2020;\nMaynez et al., 2020). This concept bears a loose\nresemblance to the phenomenon of hallucination\nobserved in human psychology. Generally, halluci-\nnations in natural language generation tasks can be\ncategorized into two primary types: intrinsic hallu-\ncination andextrinsic hallucination (Huang et al.,\n2021; Li et al., 2022b; Ji

## Using OpenAI

In [13]:
import os
openai_api_key = os.environ["OPENAI_API_KEY"]

In [33]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template(
    """Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}"""
)

llm = ChatOpenAI()
document_chain = create_stuff_documents_chain(llm, prompt)
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [34]:
response = retrieval_chain.invoke({"input": "What is hallucination?"})

In [35]:
response["answer"]

'Hallucination, in the context of natural language generation tasks, refers to a phenomenon where the generated content appears nonsensical or unfaithful to the provided source content.'

# Hull university 

In [7]:
load_hull_data = TextLoader("data.txt")
hull_data_pages = load_hull_data.load_and_split()

In [8]:
hull_document = ""
for hull_page in hull_data_pages:
    hull_document += hull_page.page_content

print(hull_document)
type(hull_document)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



str

### Create Index

In [28]:

# with torch.device(device):
#     RAG.index(collection=[hull_document], 
#               index_name="hull_data", 
#               max_document_length=512, 
#               split_documents=True, 
#               use_faiss=False)

### Load index

In [32]:
index_name = "hull_data"
results = RAG.search(query="How much is AI and DS school feels?", k=1, index_name=index_name)
print(results)

[{'content': 'The design of the new building offers an immersive, contemporary, and high-quality digital setting whilst keeping its environmental impact to a minimum.Dr Kevin Pimbblet, Director of the new Centre, said: “The past 12 months has shown how quickly the field of Artificial Intelligence can develop. With some jobs and careers now in danger of being replaced with this technology, let alone the ethical use of it, one thing is for certain: it is more important than ever that the nation has appropriately skilled graduates to be able to understand and navigate the full breadth of the technology underpinning the next industrial revolution.”\n\nCrucially, through knowledge exchange, DAIM’s ambitions encompass the provision of exemplary service within the University, and beyond to the public sector, that will enhance mutual goals and tackle complex industry issues.\n\nDAIM will become a portal for business partnerships and will deliver dynamic inter-disciplinary collaborations and ex

In [15]:
retriever = RAG.as_langchain_retriever(k=3)

In [35]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
import gradio as gr

In [36]:
prompt = ChatPromptTemplate.from_template(
    """Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}"""
)

llm = ChatOpenAI()
document_chain = create_stuff_documents_chain(llm, prompt)
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [26]:
# response = retrieval_chain.invoke({"input": "Who Is Dr Temitayo Fagbola?"})
response = retrieval_chain.invoke({"input": "In one sentence, who is dr brian?"})

In [27]:
response["answer"]

'Dr. Brian May is a highly accomplished musician, passionate astrophysicist, and lecturer in Mechanical Engineering at the University of Hull.'

# Scrap Data from URL

In [ ]:
# def get_sitemap_urls(sitemap_url):
#     response = requests.get(sitemap_url)
#     sitemap = response.content
#     root = ET.fromstring(sitemap)
#     urls = [url.text for url in root.findall('.//{http://www.sitemaps.org/schemas/sitemap/0.9}loc')]
#     return urls

# # Get URLs from sitemap
# sitemap_url = 'https://www.hull.ac.uk/sitemap.xml'
# urls = get_sitemap_urls(sitemap_url)

# from langchain.document_loaders import UnstructuredURLLoader
# while tf.device(device):
#     loaders = UnstructuredURLLoader(urls=urls)
#     data = loaders.load()

# Save & Load Data

In [ ]:
# import json
# # Prepare data for saving
# data_to_save = [{'page_content': doc.page_content, 'metadata': doc.metadata} for doc in data]

# # Save data to a JSON file
# with open('data.json', 'w') as file:
#     json.dump(data_to_save, file)

# data_str = '\n'.join(doc.page_content for doc in data)

# # Save the string to a text file
# with open('data.txt', 'w') as file:
#     file.write(data_str)

# Gradio

In [38]:
import gradio as gr
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader
from ragatouille import RAGPretrainedModel
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.prompts import ChatPromptTemplate
from langchain.llms import OpenAI
from langchain_openai import ChatOpenAI

# Load and split the data
load_hull_data = TextLoader("data.txt")
hull_data_pages = load_hull_data.load_and_split()

hull_document = ""
for hull_page in hull_data_pages:
    hull_document += hull_page.page_content

# Create a RAG model
# RAG = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")

# Specify the index name
index_name = "hull_data"

# Create a retriever
retriever = RAG.as_langchain_retriever(k=3)

# Create a prompt template
prompt = ChatPromptTemplate.from_template(
    """Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}"""
)

# Create a language model
llm = ChatOpenAI()

# Create a document chain and retrieval chain
document_chain = create_stuff_documents_chain(llm, prompt)
retrieval_chain = create_retrieval_chain(retriever, document_chain)

def ask_question(question):
    response = retrieval_chain.invoke({"input": question})
    return response["answer"]

iface = gr.Interface(
    fn=ask_question,
    inputs=gr.Textbox(lines=2, placeholder="Ask a question..."),
    outputs="text",
    title="Question Answering with Langchain and RAGatouille",
    description="Type your question and press Enter to get an answer based on the loaded data.",
)

iface.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
